In [1]:
import numpy as np
from tqdm import tqdm
from PIL import Image
import json
import os
import copy

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

### 데이터셋 생성

In [2]:
class Scalp_Health_Dataset(Dataset) :
    def __init__(self, image_path_list, label_list) : # 용량을 고려해 이미지는 경로만 받는걸로
        self.image_path_list = image_path_list
        self.label_list = label_list
    def __len__(self) : 
        return len(self.image_path_list)

    def __getitem__(self, index) : # 한 개의 데이터 가져오는 함수
        # 224 X 224로 전처리
        
        to_tensor = transforms.ToTensor()
        img = to_tensor(Image.open(self.image_path_list[index]).convert('RGB'))
        img.resize_(3, 224, 224)
        img = torch.divide(img, 255.0) # 텐서로 변경 후 이미지 리사이징하고 각 채널을 0~1 사이의 값으로 만들어버림
        
        # 출력값은 각 항목(Value1~Value6)은 3으로 나눈 뒤 출력. (각 항목의 최대값이 3)
        # 모델에서 출력값을 얻고 3씩 곱해주면 된다
        
        label = torch.tensor(self.label_list[index]/3.0)
        
        return img, label


In [3]:
def make_dataset(dataset_path, category) : # root_path + '/Train'이나 root_path + '/Label'을 받음
    
    
    image_group_folder_path = dataset_path + '/Image'
    label_group_folder_path = dataset_path + '/Label'
    
    ori_label_folder_list = os.listdir(label_group_folder_path) # '[라벨]피지과다_3.중증' 등 폴더명 알기
    
    label_folder_list = []
    
    for i in range(len(ori_label_folder_list)) :
        if ori_label_folder_list[i] != '.DS_Store' : # '.DS_Store'가 생성되었을 수 있으니 폴더 목록에서 제외
            label_folder_list.append(ori_label_folder_list[i])
    
    image_path_list = []
    label_list = []
    
    desc_str = category + "_make_dataset"
    
    for i in tqdm(range(len(label_folder_list)), desc = desc_str) :
                  
        label_folder_path = label_group_folder_path + "/" + label_folder_list[i]
        
        # label_folder_list에서 '라벨'을 '원천'으로 만 바꿔도 image파일들이 들어있는 폴더명으로 만들 수 있다
        image_folder_path = image_group_folder_path + "/" + label_folder_list[i].replace('라벨', '원천')
        
        json_list = os.listdir(label_folder_path) # json파일 목록 담기

        for j in range(len(json_list)) : 
            json_file_path = label_folder_path + '/' + json_list[j]

            with open(json_file_path, "r", encoding="utf8") as f: 
                contents = f.read() # string 타입 
                json_content = json.loads(contents) # 딕셔너리로 저장

            image_file_name = json_content['image_file_name'] # 라벨 데이터에 이미지 파일의 이름이 들어있다
            
            image_file_path = image_folder_path + "/" + image_file_name

            y_true = []
            y_true.append(int(json_content['value_1']))
            y_true.append(int(json_content['value_2']))
            y_true.append(int(json_content['value_3']))
            y_true.append(int(json_content['value_4']))
            y_true.append(int(json_content['value_5']))
            y_true.append(int(json_content['value_6']))

            y_true = np.asarray(y_true)

            image_path_list.append(image_file_path)
            label_list.append(y_true)

    return image_path_list, label_list
    
    

In [4]:
def get_dataset(root_path) : 
    
    Train_image_path_list, Train_label_list = make_dataset(root_path + '/Train', "Train")
    Test_image_path_list, Test_label_list = make_dataset(root_path + '/Test', "Test")
    
    Train_Scalp_Health_Dataset = Scalp_Health_Dataset(Train_image_path_list, Train_label_list)
    Test_Scalp_Health_Dataset = Scalp_Health_Dataset(Test_image_path_list, Test_label_list)
    
    return Train_Scalp_Health_Dataset, Test_Scalp_Health_Dataset
    
    

### 학습시키는 메소드

In [5]:
def train_model(model, criterion, optimizer, epochs, data_loader, device) :
    # 학습 -> 성능 측정
    
    pred_loss = -1.0
    checkpoint_model = 0
    
    pbar = tqdm(range(epochs), desc="training", mininterval=0.01)
    

    for epoch in pbar:  # loop over the dataset multiple times
            
        if epoch == int(epochs * 0.5) or int(epochs * 0.75) : # 31, 61번 째 에포크에서 학습률을 10배 줄임
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / 10.0
            
        
        for inputs, labels in data_loader:

            # get the inputs
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # print statistics
            pbar_str = "training, [loss = %.4f]" % loss.item()
            pbar.set_description(pbar_str)
            
            # 체크포인트
            # 배치 단위로 학습시킬 때마다 체크포인트 저장 여부 확인
            if pred_loss == -1.0 :
                pred_loss = loss
                checkpoint_model = copy.deepcopy(model)
            elif torch.lt(loss, pred_loss) == True : # loss를 더 줄였으면
                pred_loss = loss
                checkpoint_model = copy.deepcopy(model)
    
    # 체크포인트에 저장했던걸 최종 모델로
    model = checkpoint_model
    
    return model

### 구현

In [6]:
root_path = '/home/ubuntu/CUAI_2021/Advanced_Minkyu_Kim/Scalp_Health_Dataset'
Train_Scalp_Health_Dataset, Test_Scalp_Health_Dataset = get_dataset(root_path)

Test_make_dataset: 100%|██████████| 24/24 [00:00<00:00, 289.84it/s]


In [7]:
# 학습에 사용
BATCH_SIZE = 64

data_loader = torch.utils.data.DataLoader(dataset=Train_Scalp_Health_Dataset, # 사용할 데이터셋
                                          batch_size=BATCH_SIZE, # 미니배치 크기
                                          shuffle=True, # 에포크마다 데이터셋 셔플할건가? 
                                          drop_last=True) # 마지막 배치가 BATCH_SIZE보다 작을 수 있다. 나머지가 항상 0일 수는 없지 않는가. 이 때 마지막 배치는 사용하지 않으려면 drop_last = True를, 사용할거면 drop_last = False를 입력한다

#### 네트워크 생성 : DenseNet_161에 classifier만 수정해서 학습시킨다

In [8]:
# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = models.densenet161(pretrained = True, memory_efficient = True).to(device)

# ImageNet으로 학습시킨 CNN은 수정하지 못하게끔
for param in model.features.parameters():
    param.requires_grad = False

In [9]:
# 모발 분류를 위한 linear model 생성 후 교체
new_classifier = nn.Sequential(
            nn.Linear(in_features=2208, out_features=6, bias=True)
        ).to(device)

for m in new_classifier.modules():
    if isinstance(m, nn.Linear) :
        nn.init.kaiming_uniform_(m.weight)
        
model.classifier = new_classifier # 교체

In [10]:
# 학습에 필요한 것들

optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum = 0.9, weight_decay = 1e-4)
EPOCHS = 300
loss = nn.MSELoss() # 5종류의 출력값을 예측하는 선형회귀 모델이라 MSE사용

#### 학습

In [ ]:
model = train_model(model, loss, optimizer, EPOCHS, data_loader, device)

training, [loss = 0.1450]:   0%|          | 0/300 [01:42<?, ?it/s]